<a href="https://colab.research.google.com/github/tbonne/IntroDataScience/blob/main/InClassNotebooks/IntroCausalAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explainability vs Causality

Here we will look at the difference between understanding how the ML model is making predictions (explainability) and what is causing the outcome (causality)


To do so we will look at a silly example where we know that the patterns picked up by the model are not causal.


## Waffle houses and divorce rates

In [ ]:
import pandas as pd
import sklearn as sk
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

Load the data (get the data from slack, and place it into your google drive folder)

In [ ]:
#load data
df_waffles = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataScience/IntroCausalAnalysis/waffles.csv")

#take a look
df_waffles.head()

Visualize the data

In [ ]:
#sort the dataframe
pd_df = df_waffles.sort_values(['Divorce']).reset_index(drop=True)

#plot by state
sns.barplot(data=pd_df, x="Loc",y="Divorce")
plt.xticks(rotation=90)

### Do whaffle houses cause divorce?

In [ ]:
#correlation
df_waffles.WaffleHouses.corr(df_waffles.Divorce)

In [ ]:
#scatter plot
sns.scatterplot(data=df_waffles, x="WaffleHouses", y="Divorce" )


Data wrangling

In [ ]:
#split these data into training and testing datasets
df_train, df_test = train_test_split(df_waffles, test_size=0.20, random_state=34)

# Build a model

Can we predict divorce rates based on:
1. population
2. marage rates (more mariage more divorce)
3. Median age at marriage
4. Number of waffle houses

In [ ]:
import statsmodels.api as sm #for running regression!
import statsmodels.formula.api as smf

#Build the model
linear_reg_model = smf.ols(formula='Divorce ~ WaffleHouses + Population + Marriage', data=df_train)

#Use the data to fit the model (i.e., find the best intercept and slope parameters)
linear_reg_results = linear_reg_model.fit()

#summary
print(linear_reg_results.summary())

### Fit the model again, this time add the South variable.

In [ ]:
#Build the model
linear_reg_model_South = smf.ols(formula='Divorce ~ WaffleHouses + Population + Marriage + South', data=df_train)

#Use the data to fit the model (i.e., find the best intercept and slope parameters)
linear_reg_model_South = linear_reg_model_South.fit()

#summary
print(linear_reg_model_South.summary())

## Let's see what feature importance suggests

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance

#split data into predictors (X) and target (y)
X = df_waffles[['WaffleHouses', 'Population', 'Marriage', 'South']]
y = df_waffles['Divorce']

#split these data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#fit linear regression
LR1 = LinearRegression()
LR1.fit(X_train, y_train)

#prediction error
mean_squared_error(LR1.predict(X_test), y_test)

#model interpretation
rel_impo = permutation_importance(LR1, X_test, y_test,n_repeats=30,random_state=0)
pd.DataFrame({"feature":X_test.columns,"importance":rel_impo.importances_mean, "sd":rel_impo.importances_std})

## Let's see what feature selection suggests

In [ ]:
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV

#split data into predictors (X) and target (y)
X = df_waffles.drop(['Divorce','Unnamed: 0','Location','Loc'], axis=1)
y = df_waffles['Divorce']

#split these data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#build a linear regression (full model)
LR1 = LinearRegression()

#fit linear regression
LR1.fit(X_train, y_train)

In [ ]:
#min number of variables/features
min_features_to_select = 1

#build the feature selection algorithm
rfecv = RFECV(estimator=LR1, step=1, cv=3,scoring='neg_mean_squared_error', min_features_to_select=min_features_to_select)

#fit the algorithm to the data
rfecv.fit(X_train, y_train)

In [ ]:
print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (mean square error?)")
plt.plot(range(min_features_to_select,
               len(rfecv.grid_scores_) + min_features_to_select),
         rfecv.grid_scores_)
plt.show()

In [ ]:
rfecv.support_

In [ ]:
X_train_reduced = X_train.iloc[:,rfecv.support_]

X_train_reduced.head(3)

In [ ]:
#get the slopes!
rfecv.estimator_.coef_

### <font color='lightblue'>Statistical confounds</font>

> Statistical confounds make it hard to determine the causal nature of the patterns we find in ML model results. We need to be careful about how we explain how a model makes predictions and the causal nature of those patterns.

> In the case of the whaffle houses and divorce rates, there are just more waffle houses in southern states.  

In [ ]:
sns.boxplot(data=df_waffles, x="South", y="WaffleHouses")

### <font color='lightblue'>Bonus</font>

Redo the exercise above this time using a more black box approach, e.g., Random Forest!

In [ ]:
from sklearn.model_selection import train_test_split

#split data into predictors (X) and target (y)
X = df_waffles[['Population','Marriage','WaffleHouses']]
y = df_waffles['Divorce']

#split these data into training and testing datasets
?

Find optimal hyperparameters

In [ ]:
?




Build the model with the optimal hyperparameters

In [ ]:
#1. build the model
RFR = ?

#2. fit the model to the data
?

#3. make predictions using the model
y_pred = ?

How well did the model perform

In [ ]:
#how well does it predict
from sklearn.metrics import mean_squared_error

?

Explain how the model is making these predictions?

In [ ]:
#What is important for prediction?
from sklearn.inspection import permutation_importance

#estimate permutation importance on the test data
perm_impo = ?

#create a dataframe with the values
df_imp = pd.DataFrame({"feature":X_test.columns,"importance":perm_impo.importances_mean, "sd":perm_impo.importances_std})
sorted(sk.metrics.SCORERS.keys())

#take a look
df_imp

In [ ]:
#plot the importance values
df_imp_all = pd.DataFrame(perm_impo.importances.transpose())
df_imp_all.columns = X_test.columns
df_imp_all_long = pd.melt(df_imp_all)
sns.barplot(data=df_imp_all_long, x="variable",y="value", ci=95)
plt.xticks(rotation=90) 


What does the model think divorce rates will change when we vary wafflehouses?

In [ ]:
#1. Create a dataframe
df_question = pd.DataFrame({'Population':X_train.Population.mean(),
                            'WaffleHouses':list(range(0,200,10)),
                            'Marriage':X_train.Marriage.mean(),
                            })
                            

#2. Use the model to make predictions
question_pred =  RFR.?

#3. add a column to the df_question
df_question['predicted_divorceRates'] = ?

#4. plot the predictions
sns.scatterplot(data=df_question, x='?',y='?')
question_pred

In my case, the random forest model has found a positive association between wafflehouses and divorce rates!

### Repeat the analysis but this time add in the variable 'South'


Data wrangling

In [ ]:
#split data into predictors (X) and target (y)
X = df_waffles[['Population','Marriage','WaffleHouses','South']]
y = df_waffles['Divorce']


#split these data into training and testing datasets
?

Build the model

In [ ]:
?

Make predictions 

In [ ]:
mean_squared_error(?)

In [ ]:
rel_impo2 = permutation_importance(?)

pd.DataFrame({"feature":X_test.columns,"importance":rel_impo2.importances_mean, "sd":rel_impo2.importances_std})

In [ ]:
df_imp_all = pd.DataFrame(rel_impo2.importances.transpose())
df_imp_all.columns = X_test.columns
df_imp_all_long = pd.melt(df_imp_all)
sns.barplot(data=df_imp_all_long, x="variable",y="value", ci=95)
plt.xticks(rotation=90)


What does the model think divorce rates will change when we vary wafflehouses?

In [ ]:
#1. Create a dataframe
df_question = pd.DataFrame({'Population':X_train.Population.mean(),
                            'WaffleHouses':list(range(0,200,10)),
                            'Marriage':X_train.Marriage.mean(),
                            'South':1
                            })
                            

#2. Use the model to make predictions
question_pred =  RFR2.?

#3. add a column to the df_question
df_question['predicted_divorceRates'] = ?

#4. plot the predictions
sns.scatterplot(data=df_question, x='?',y='?')
question_pred

In this case, the random forest model has still found a positive association between wafflehouses and divorce rates!